In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process


songs_db = pd.read_csv("data.csv")

songs_db.head()

C:\Users\user\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [2]:
songs_db['artists'] = songs_db['artists'].str.strip("[]").str.replace("'", "")

In [3]:
songs_db["combined_features"] = songs_db["artists"] + ' ' + songs_db["name"]

In [4]:
songs_db.isnull().sum()

valence              0
year                 0
acousticness         0
artists              0
danceability         0
duration_ms          0
energy               0
explicit             0
id                   0
instrumentalness     0
key                  0
liveness             0
loudness             0
mode                 0
name                 0
popularity           0
release_date         0
speechiness          0
tempo                0
combined_features    0
dtype: int64

In [5]:
songs_db.drop(columns=["duration_ms", "energy", "tempo", "mode", "explicit", "speechiness", "acousticness"], inplace=True)

id_to_index = {id_: idx for idx, id_ in enumerate(songs_db['id'].unique())}

songs_db['id_index'] = songs_db['id'].map(id_to_index)

songs_db = songs_db.drop(columns=['id'])

In [6]:
songs_db.head()

,valence,year,artists,danceability,instrumentalness,key,liveness,loudness,name,popularity,release_date,combined_features,id_index
0,0.0594,1921,"Sergei Rachmaninoff, James Levine, Berliner Ph...",0.279,0.878000,10,0.665,-20.096,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,"Sergei Rachmaninoff, James Levine, Berliner Ph...",0
1,0.9630,1921,Dennis Day,0.819,0.000000,7,0.160,-12.441,Clancy Lowered the Boom,5,1921,Dennis Day Clancy Lowered the Boom,1
2,0.0394,1921,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.328,0.913000,3,0.101,-14.850,Gati Bali,5,1921,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,2
3,0.1650,1921,Frank Parker,0.275,0.000028,5,0.381,-9.316,Danny Boy,3,1921,Frank Parker Danny Boy,3
4,0.2530,1921,Phil Regan,0.418,0.000002,3,0.229,-10.096,When Irish Eyes Are Smiling,2,1921,Phil Regan When Irish Eyes Are Smiling,4


In [7]:
df_musics = songs_db[['id_index', 'combined_features']].astype({'id_index': 'int32', 'combined_features': 'str'})

df_dancebilities = songs_db[['id_index', 'danceability']].astype({'id_index': 'int32', 'danceability': 'float32'})

df_popularity = songs_db[['id_index', 'popularity']].astype({'id_index': 'int32', 'popularity': 'int32'})

In [28]:
def recommend(name):
    result = process.extractOne(name, df_musics['combined_features'])
    if result:
        print(result)
    else:
        print("No match found.")

def recommendByDancebility(danceability, df_dancebilities):
    df_dancebilities['danceability'] = pd.to_numeric(df_dancebilities['danceability'], errors='coerce')
    
    closest_danceability = df_dancebilities['danceability'].sub(danceability).abs().idxmin()
    closest_match = df_dancebilities.loc[closest_danceability]

    result = process.extractOne(str(int(closest_match["id_index"])), str(df_musics['id_index']))
    
    if result:
        print(result)
    else:
        print("No match found.")

In [29]:
recommend("kanye")
recommendByDancebility(0.8, df_dancebilities)


('Kanye West Through The Wire', 90, 16444)
('2', 90)
